In [ ]:
using Images, Colors, Distributions

function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    dx=(2.0*radio)/(1.0*ancho)  #el ancho que representa un pixel
    dy=(2.0im*radio)/(1.0*alto) #el alto que representa un pixel
    z0 = centro + radio*(1.0im - 1.0) #el primer valor de la muestra es la esquina superior izquierda
    return [z0 + (x*dx) - (y*dy) for y in 0:alto-1, x in 0:ancho-1]
end

# Paletas de colores custom

Hasta ahora la eleccion de colores solo muestra el tiempo de escape.
Nos recordamos que el espacio de colores es de dimensión 3, asi que podemos codificar informacion de tipo tridimensional en los colores que elegimos. El tiempo de escape es una *informacion unidimensional*.

Los dos grados de libertad que tenemos nos permiten elegir paletas que se adecuen a la estética deseada.

En este ejemplo obtendremos una paleta de colores a partir de una caminata aletatoria sobre una imagen. La función ``moneda`` regresa un numero entero entre -N y N distribuido de manera uniforme. La función ``paleta`` carga la imagen en la ruta indicada en el parametro ``mi_foto`` y realiza una caminata aleatoria con la distribución dada por ``moneda(paso)`` hasta llenar un arreglo de colores del ``tamaño`` indicado.

In [ ]:
function moneda(N)
    ceil(Int64, round(rand(Uniform(-N-0.499, N+0.499))))
end

function paleta(mi_foto, paso, tamaño)
    aesthetics = load(mi_foto)
    alto, ancho =size(aesthetics)
    x0 = ceil(Int64, rand(Uniform(0,ancho)))
    #x0 = ceil(Int64, ancho/3)
    y0 = ceil(Int64, rand(Uniform(0,alto)))
    #y0 = ceil(Int64, alto/2)
    paleta = zeros(RGB{N0f8}, tamaño)
    for i = 1:length(paleta)
        paleta[i] = aesthetics[mod(y0,alto)+1,mod(x0,ancho)+1]
        x0 = x0 + moneda(paso)
        y0 = y0 + moneda(paso)
    end
    return paleta
end

La imagen que usaremos en el ejemplo es la siguiente:

In [ ]:
load("aesthetic.jpg")

Cargaremos una paleta en ``mis_colores`` para los ejemplos siguientes.

De nuevo, la trampa será el complemento de un disco centrado en el origen.

In [ ]:
mis_colores = paleta("aesthetic.jpg", 10, 1000)

In [ ]:
function mi_trampa_custom(Z, F, paleta, R=3.0, TOPE=1000)
    cont = 0
    while cont <= TOPE
        if(abs(Z) > R) 
            n = 1 + mod(cont, length(paleta))
            return paleta[n]
        end
        Z = F(Z)
        cont = cont + 1
    end
    return RGB{Float64}(0.0, 0.0, 0.0) #negro
end

In [ ]:
function mandelbrot_custom(paleta, R=2.0, TOPE=1000, centro=0.0, radio=2.0, ancho=1000, alto=1000)
    broadcast(C -> mi_trampa_custom(0.0, z -> z^2 + C, paleta, R, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
mandelbrot_custom(mis_colores)

In [ ]:
mandelbrot_custom(mis_colores, 2.0, 500, -0.75, 1.4)

In [ ]:
mandelbrot_custom(mis_colores, 2.0, 1000, -0.0232+0.999im, 0.0005)

## Suavizado de iteraciones

La versión anterior de la trampa cuenta el número de iteraciones que le toma al punto evaluado en llegar a la región de escape, por lo que el *tiempo de escape* es siempre un numero entero.

Modificaremos el algoritmo para que el resultado sea un numero de punto flotante. El color que se escogera será el corespondiente a una interpolacion del color asociado a la parte entera de este tiempo suavizado y el siguiente en el arreglo de colores deseado.

Por ejemplo, en lugar de aportar ``+1`` a la cuenta, cada iteración aportará una cantidad proporcional a la distancia (esférica ``sph_dist`` o en la carta al infinito ``inf_dist``) entre el punto actual y el anterior. 

Es posible modificar el algoritmo para dibujar el conjunto de Mandelbrot usando la función de Green de su complemento. Esa modificación se deja al usuario :)

In [ ]:
sph_dist(Z, W) = (2.0*abs(Z-W))/sqrt((1.0 + abs(Z)^2)*(1.0 + abs(W)^2))

inf_dist(Z, W) = abs(Z-W)/abs(Z*W)

function mi_trampa_suave(Z, F, paleta, R=3.0, TOPE=1000)
    cont = 0
    S = 0.0
    W = F(Z)
    while cont <= TOPE
        if(abs(Z) > R)
            n = floor(Int64, S)
            ϵ = S - n
            n = mod(n, length(paleta)-1)
            return (1.0-ϵ)*paleta[n+1] + (ϵ)*paleta[n+2]
        end
        S += 1.0 - exp(-0.2*inf_dist(Z,W))
        Z = W
        W = F(Z)
        cont += 1
    end
    return RGB{Float64}(0.0, 0.0, 0.0) #negro
end

In [ ]:
function mandelbrot_custom_suave(paleta, R=2.0, TOPE=1000, centro=0.0, radio=2.0, ancho=1000, alto=1000)
    broadcast(C -> mi_trampa_suave(0.0, z -> z^2 + C, paleta, R, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
mandelbrot_custom_suave(mis_colores)

In [ ]:
mandelbrot_custom_suave(mis_colores, 2.0, 500, -0.75, 1.4)

In [ ]:
mandelbrot_custom_suave(mis_colores, 2.0, 1000, -0.0232+0.999im, 0.0005)